In [1]:
import git
from pathlib import Path
import os

ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

DATASET = "coco"
FINAL_DATA_NAME = 'coco-indoor-wavelet'
CONSTANT_SAMPLE_SIZE = int(1e5)
RAW_DATA_SUFFIX = "coco-cropped-Indoor"
SAVE_DF = False

data_dir = os.path.join(ROOT_DIR, 'raw-data','coco')
file_list = [os.path.join(data_dir, filename) for filename in os.listdir(data_dir)]
file_names = os.listdir(data_dir)
data_dir
BATCH_NUM = None

In [2]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from transform import *
os.chdir(os.path.join(ROOT_DIR, "dataset-preparation"))
freq_df = pd.read_csv(os.path.join(ROOT_DIR, "transformed-data", f"master-frequency-map.csv"), index_col= ["dataset", "transform", "group"])


In [3]:
file_list = [os.path.join(data_dir, f"toy-{RAW_DATA_SUFFIX}", filename) for filename in os.listdir(data_dir)]
file_names = os.listdir(os.path.join(data_dir, f"toy-{RAW_DATA_SUFFIX}"))
file_names[:5]

['000000182611.jpg',
 '000000479126.jpg',
 '000000304396.jpg',
 '000000231339.jpg',
 '000000377393.jpg']

In [4]:
'''Assuming No batching is required. Not applicable for agriVision'''

# data_dir = os.path.join(ROOT_DIR, "raw-data", "agriVision", "full-agriVision-RGB-cleaned")

# for channel in ['red', 'blue', 'green', 'gray', 'infrared']:

#     channel_fr = convert_to_fourier_basis(data_dir, channel, debug = True)
#     pd.to_pickle(channel_fr, os.path.join(ROOT_DIR, "transformed-data", f"full-agriVision-fourier-{channel}-df.pickle"))

#     min_group, max_group = 2, max(channel_fr['band'])
#     group_data_map = dict()
#     group_data_map_size = dict()
#     for group in np.arange(min_group, max_group + 1):
#         data = channel_fr[(channel_fr['band'] == group)]['data'].iloc[0]
#         group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)] 
#         group_data_map_size[group] = data.size
    
#     pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}.pickle"))
#     pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}-size.pickle"))
    

'Assuming No batching is required. Not applicable for agriVision'

In [5]:
'''To split large dataset into many batches, only needs to be run once'''
# k = 10000
# target_dir = os.path.join(ROOT_DIR, 'raw-data', 'agriVision') # Where the batch{i} folders will be created
# directorySplit(folder_dir = data_dir, target_dir = target_dir, name = RAW_DATA_SUFFIX, k = k)
# print(f"{len(file_names)//k} batches created" )

'To split large dataset into many batches, only needs to be run once'

In [4]:
'''Show all subsets of data in raw data folder that have already been created'''
print(''.join([x+"\n" for x in os.listdir(data_dir) if x.__contains__(RAW_DATA_SUFFIX)]))


toy-coco-cropped-Indoor
._toy-coco-cropped-Indoor



In [8]:
def npz_opener_pickle(path):
    image = Image.open(path).convert('RGB')
    return np.array(image)

# Wavelet

In [6]:
FINAL_DATA_NAME = 'coco-indoor-wavelet'
if BATCH_NUM is None:
    batch_dir = os.path.join(ROOT_DIR, "raw-data", "coco", f"toy-{RAW_DATA_SUFFIX}")
else:
    batch_dir = os.path.join(ROOT_DIR, "raw-data", "coco", f"batch{BATCH_NUM}-{RAW_DATA_SUFFIX}")

In [9]:
TRANSFORM = "wavelet"
channel = "red"

for orientation_label in ['horizontal', 'vertical']:
    FINAL_DATA_NAME_ORIENTED = f"{FINAL_DATA_NAME}-{orientation_label}"
    orientation_code = 'H' if orientation_label == 'horizontal' else 'V'

    channel_wv = convert_to_wavelet_basis(batch_dir, channel, debug=True, image_opener=npz_opener_pickle)
    channel_wv['data'] = channel_wv['data'].apply(lambda x: x.astype(np.float32))

    if SAVE_DF:
        if BATCH_NUM is None:
            pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/{FINAL_DATA_NAME_ORIENTED}-{channel}-df.pickle"))
        else:
            pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/batch{BATCH_NUM}{FINAL_DATA_NAME_ORIENTED}-{channel}-df.pickle"))

    min_group, max_group = 2, max(channel_wv['layer'])
    group_data_map = dict()
    group_data_map_size = dict()

    for group in np.arange(min_group, max_group + 1):
        data = channel_wv[(channel_wv['orientation'] == orientation_code) & (channel_wv['layer'] == group)]['data'].iloc[0]
        group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)]
        group_data_map_size[group] = data.size

        freq_df.loc[DATASET, TRANSFORM, group] = channel_wv[(channel_wv['layer'] == group) & (channel_wv['orientation'] == orientation_code)]['frequency'].iloc[0]

    print(f"printing {orientation_code}")
    freq_df.to_csv(os.path.join(ROOT_DIR, "transformed-data", "master-frequency-map.csv"))

    if BATCH_NUM is None:
        pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME_ORIENTED}-{channel}.pickle"))
        pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME_ORIENTED}-{channel}-size.pickle"))
    else:
        pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME_ORIENTED}-{channel}.pickle"))
        pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME_ORIENTED}-{channel}-size.pickle"))

    print(group_data_map)

    del channel_wv
    del group_data_map
    del group_data_map_size


9 layers being used


100%|██████████| 1604/1604 [00:08<00:00, 184.57it/s]


printing H
{np.int64(2): array([-18543.719, -17717.684, -17385.676, ...,  18247.875,  20605.406,
        20710.25 ], shape=(1604,), dtype=float32), np.int64(3): array([-10430.547, -10339.836,  -9587.359, ...,   9851.672,  11157.68 ,
        12393.016], shape=(6416,), dtype=float32), np.int64(4): array([-6697.2812, -6542.2188, -6271.875 , ...,  5856.5625,  5998.3594,
        6574.3125], shape=(25664,), dtype=float32), np.int64(5): array([-3342.8438, -3152.5625, -3013.7812, ...,  3164.4375,  3244.6875,
        3248.1562], shape=(100000,), dtype=float32), np.int64(6): array([-1798.5   , -1714.75  , -1641.875 , ...,  1577.875 ,  1651.625 ,
        1701.1875], shape=(100000,), dtype=float32), np.int64(7): array([-906.5  , -841.25 , -821.5  , ...,  796.625,  820.625,  869.125],
      shape=(100000,), dtype=float32), np.int64(8): array([-469.5 , -418.  , -403.75, ...,  397.25,  411.  ,  461.5 ],
      shape=(100000,), dtype=float32), np.int64(9): array([-250.5, -215.5, -208. , ...,  206.5,  2

100%|██████████| 1604/1604 [00:02<00:00, 777.64it/s]


printing V
{np.int64(2): array([-21876.5  , -17983.02 , -15719.383, ...,  15973.383,  17924.586,
        21353.223], shape=(1604,), dtype=float32), np.int64(3): array([-10322.133, -10310.031, -10190.125, ...,   9512.305,  10047.492,
        11434.602], shape=(6416,), dtype=float32), np.int64(4): array([-6386.3594, -6183.672 , -5816.078 , ...,  6326.1094,  6453.9375,
        6739.25  ], shape=(25664,), dtype=float32), np.int64(5): array([-3333.3438, -3300.25  , -3275.1562, ...,  3132.9062,  3146.1562,
        3290.3125], shape=(100000,), dtype=float32), np.int64(6): array([-1736.75  , -1664.375 , -1646.0625, ...,  1593.25  ,  1640.    ,
        1783.4375], shape=(100000,), dtype=float32), np.int64(7): array([-881.625, -825.125, -796.5  , ...,  779.75 ,  822.   ,  988.875],
      shape=(100000,), dtype=float32), np.int64(8): array([-460.75, -414.25, -399.  , ...,  395.5 ,  408.75,  484.  ],
      shape=(100000,), dtype=float32), np.int64(9): array([-253. , -213. , -203. , ...,  204.5,  2

In [11]:
TRANSFORM = "wavelet"
channel = "green"

for orientation_label in ['horizontal', 'vertical']:
    FINAL_DATA_NAME_ORIENTED = f"{FINAL_DATA_NAME}-{orientation_label}"
    orientation_code = 'H' if orientation_label == 'horizontal' else 'V'

    channel_wv = convert_to_wavelet_basis(batch_dir, channel, debug=True, image_opener=npz_opener_pickle)
    channel_wv['data'] = channel_wv['data'].apply(lambda x: x.astype(np.float32))

    if SAVE_DF:
        if BATCH_NUM is None:
            pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/{FINAL_DATA_NAME_ORIENTED}-{channel}-df.pickle"))
        else:
            pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/batch{BATCH_NUM}{FINAL_DATA_NAME_ORIENTED}-{channel}-df.pickle"))

    min_group, max_group = 2, max(channel_wv['layer'])
    group_data_map = dict()
    group_data_map_size = dict()

    for group in np.arange(min_group, max_group + 1):
        data = channel_wv[(channel_wv['orientation'] == orientation_code) & (channel_wv['layer'] == group)]['data'].iloc[0]
        group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)]
        group_data_map_size[group] = data.size

        freq_df.loc[DATASET, TRANSFORM, group] = channel_wv[(channel_wv['layer'] == group) & (channel_wv['orientation'] == orientation_code)]['frequency'].iloc[0]

    print(f"printing {orientation_code}")
    freq_df.to_csv(os.path.join(ROOT_DIR, "transformed-data", "master-frequency-map.csv"))

    if BATCH_NUM is None:
        pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME_ORIENTED}-{channel}.pickle"))
        pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME_ORIENTED}-{channel}-size.pickle"))
    else:
        pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME_ORIENTED}-{channel}.pickle"))
        pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME_ORIENTED}-{channel}-size.pickle"))

    print(group_data_map)

    del channel_wv
    del group_data_map
    del group_data_map_size


9 layers being used


100%|██████████| 1604/1604 [00:02<00:00, 756.02it/s]


printing H
{np.int64(2): array([-18103.84 , -16203.844, -14760.898, ...,  18247.875,  18295.133,
        20574.61 ], shape=(1604,), dtype=float32), np.int64(3): array([-11018.328 ,  -9731.242 ,  -9676.281 , ...,   9518.023 ,
         9654.5625,  11885.25  ], shape=(6416,), dtype=float32), np.int64(4): array([-7052.7344, -6691.1094, -6679.5625, ...,  6826.3438,  6920.8906,
        6970.7656], shape=(25664,), dtype=float32), np.int64(5): array([-3414.6562, -3412.75  , -3394.8438, ...,  3153.875 ,  3187.125 ,
        3244.6875], shape=(100000,), dtype=float32), np.int64(6): array([-1861.9375, -1729.625 , -1689.75  , ...,  1608.    ,  1664.6875,
        1746.3125], shape=(100000,), dtype=float32), np.int64(7): array([-927.375, -832.875, -797.   , ...,  807.5  ,  819.875,  881.625],
      shape=(100000,), dtype=float32), np.int64(8): array([-471.75, -418.5 , -404.  , ...,  398.  ,  411.25,  466.  ],
      shape=(100000,), dtype=float32), np.int64(9): array([-252.5, -217. , -208.5, ...,  207

100%|██████████| 1604/1604 [00:02<00:00, 744.70it/s]


printing V
{np.int64(2): array([-20436.48 , -18298.633, -14272.234, ...,  14553.844,  18239.445,
        20241.312], shape=(1604,), dtype=float32), np.int64(3): array([-11019.3125, -11007.883 , -10331.539 , ...,   9000.555 ,
         9104.016 ,   9221.461 ], shape=(6416,), dtype=float32), np.int64(4): array([-6242.1562, -6073.0156, -6002.875 , ...,  5466.1562,  6075.0156,
        6503.0156], shape=(25664,), dtype=float32), np.int64(5): array([-3425.2188, -3372.9688, -3355.4688, ...,  3059.8125,  3455.2812,
        3690.2188], shape=(100000,), dtype=float32), np.int64(6): array([-1746.6875, -1647.875 , -1607.3125, ...,  1622.875 ,  1684.    ,
        1771.625 ], shape=(100000,), dtype=float32), np.int64(7): array([-903.   , -816.375, -795.25 , ...,  787.25 ,  805.5  ,  983.375],
      shape=(100000,), dtype=float32), np.int64(8): array([-458.75, -415.25, -402.  , ...,  400.75,  414.25,  480.  ],
      shape=(100000,), dtype=float32), np.int64(9): array([-252.5, -214. , -205. , ...,  205

In [12]:
TRANSFORM = "wavelet"
channel = "blue"

for orientation_label in ['horizontal', 'vertical']:
    FINAL_DATA_NAME_ORIENTED = f"{FINAL_DATA_NAME}-{orientation_label}"
    orientation_code = 'H' if orientation_label == 'horizontal' else 'V'

    channel_wv = convert_to_wavelet_basis(batch_dir, channel, debug=True, image_opener=npz_opener_pickle)
    channel_wv['data'] = channel_wv['data'].apply(lambda x: x.astype(np.float32))

    if SAVE_DF:
        if BATCH_NUM is None:
            pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/{FINAL_DATA_NAME_ORIENTED}-{channel}-df.pickle"))
        else:
            pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/batch{BATCH_NUM}{FINAL_DATA_NAME_ORIENTED}-{channel}-df.pickle"))

    min_group, max_group = 2, max(channel_wv['layer'])
    group_data_map = dict()
    group_data_map_size = dict()

    for group in np.arange(min_group, max_group + 1):
        data = channel_wv[(channel_wv['orientation'] == orientation_code) & (channel_wv['layer'] == group)]['data'].iloc[0]
        group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)]
        group_data_map_size[group] = data.size

        freq_df.loc[DATASET, TRANSFORM, group] = channel_wv[(channel_wv['layer'] == group) & (channel_wv['orientation'] == orientation_code)]['frequency'].iloc[0]

    print(f"printing {orientation_code}")
    freq_df.to_csv(os.path.join(ROOT_DIR, "transformed-data", "master-frequency-map.csv"))

    if BATCH_NUM is None:
        pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME_ORIENTED}-{channel}.pickle"))
        pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME_ORIENTED}-{channel}-size.pickle"))
    else:
        pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME_ORIENTED}-{channel}.pickle"))
        pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME_ORIENTED}-{channel}-size.pickle"))

    print(group_data_map)

    del channel_wv
    del group_data_map
    del group_data_map_size


9 layers being used


100%|██████████| 1604/1604 [00:02<00:00, 737.65it/s]


printing H
{np.int64(2): array([-18660.398, -13917.836, -13476.832, ...,  18541.96 ,  18543.102,
        19479.684], shape=(1604,), dtype=float32), np.int64(3): array([-11494.656, -10832.461, -10765.508, ...,  10922.93 ,  11049.117,
        11586.109], shape=(6416,), dtype=float32), np.int64(4): array([-6689.4375, -6631.5156, -6622.9062, ...,  6231.25  ,  7058.797 ,
        7156.297 ], shape=(25664,), dtype=float32), np.int64(5): array([-3415.125 , -3306.7188, -3273.0625, ...,  3212.4688,  3222.5   ,
        3244.6875], shape=(100000,), dtype=float32), np.int64(6): array([-1751.875 , -1711.75  , -1649.0625, ...,  1587.    ,  1642.5625,
        1719.3125], shape=(100000,), dtype=float32), np.int64(7): array([-905.375, -814.75 , -786.875, ...,  778.   ,  812.625,  889.375],
      shape=(100000,), dtype=float32), np.int64(8): array([-469.5 , -414.5 , -399.5 , ...,  392.25,  408.5 ,  485.  ],
      shape=(100000,), dtype=float32), np.int64(9): array([-250.5, -215.5, -207. , ...,  206. ,  2

100%|██████████| 1604/1604 [00:02<00:00, 765.17it/s]


printing V
{np.int64(2): array([-18438.293, -15851.887, -15436.078, ...,  15699.289,  16376.777,
        19879.383], shape=(1604,), dtype=float32), np.int64(3): array([-12946.625, -12442.297, -11031.656, ...,   9263.578,   9427.094,
         9620.664], shape=(6416,), dtype=float32), np.int64(4): array([-6277.078 , -5966.5156, -5947.4062, ...,  5700.453 ,  6211.047 ,
        6453.6875], shape=(25664,), dtype=float32), np.int64(5): array([-3380.0312, -3201.1562, -3188.2812, ...,  3354.125 ,  3424.7188,
        3625.75  ], shape=(100000,), dtype=float32), np.int64(6): array([-1765.375 , -1691.125 , -1644.25  , ...,  1623.875 ,  1658.6875,
        1802.3125], shape=(100000,), dtype=float32), np.int64(7): array([-896.75 , -812.875, -786.   , ...,  785.   ,  808.625,  917.625],
      shape=(100000,), dtype=float32), np.int64(8): array([-461.5 , -411.25, -393.75, ...,  396.  ,  412.  ,  473.  ],
      shape=(100000,), dtype=float32), np.int64(9): array([-252.5, -212.5, -204. , ...,  203.5,  2

In [13]:
TRANSFORM = "wavelet"
channel = "gray"

for orientation_label in ['horizontal', 'vertical']:
    FINAL_DATA_NAME_ORIENTED = f"{FINAL_DATA_NAME}-{orientation_label}"
    orientation_code = 'H' if orientation_label == 'horizontal' else 'V'

    channel_wv = convert_to_wavelet_basis(batch_dir, channel, debug=True, image_opener=npz_opener_pickle)
    channel_wv['data'] = channel_wv['data'].apply(lambda x: x.astype(np.float32))

    if SAVE_DF:
        if BATCH_NUM is None:
            pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/{FINAL_DATA_NAME_ORIENTED}-{channel}-df.pickle"))
        else:
            pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/batch{BATCH_NUM}{FINAL_DATA_NAME_ORIENTED}-{channel}-df.pickle"))

    min_group, max_group = 2, max(channel_wv['layer'])
    group_data_map = dict()
    group_data_map_size = dict()

    for group in np.arange(min_group, max_group + 1):
        data = channel_wv[(channel_wv['orientation'] == orientation_code) & (channel_wv['layer'] == group)]['data'].iloc[0]
        group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)]
        group_data_map_size[group] = data.size

        freq_df.loc[DATASET, TRANSFORM, group] = channel_wv[(channel_wv['layer'] == group) & (channel_wv['orientation'] == orientation_code)]['frequency'].iloc[0]

    print(f"printing {orientation_code}")
    freq_df.to_csv(os.path.join(ROOT_DIR, "transformed-data", "master-frequency-map.csv"))

    if BATCH_NUM is None:
        pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME_ORIENTED}-{channel}.pickle"))
        pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME_ORIENTED}-{channel}-size.pickle"))
    else:
        pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME_ORIENTED}-{channel}.pickle"))
        pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME_ORIENTED}-{channel}-size.pickle"))

    print(group_data_map)

    del channel_wv
    del group_data_map
    del group_data_map_size


9 layers being used


100%|██████████| 1604/1604 [00:03<00:00, 510.69it/s]


printing H
{np.int64(2): array([-18050.055, -15122.361, -13620.761, ...,  17984.03 ,  18246.05 ,
        20488.273], shape=(1604,), dtype=float32), np.int64(3): array([-10895.84 ,  -9433.887,  -9411.363, ...,   9435.939,   9520.618,
        12001.73 ], shape=(6416,), dtype=float32), np.int64(4): array([-6685.491 , -6638.9683, -6432.349 , ...,  6133.2397,  6484.7754,
        6843.443 ], shape=(25664,), dtype=float32), np.int64(5): array([-3341.1008, -3244.1067, -3210.0337, ...,  3124.6318,  3184.0579,
        3244.363 ], shape=(100000,), dtype=float32), np.int64(6): array([-1725.6556, -1684.93  , -1659.922 , ...,  1561.0361,  1632.9418,
        1700.681 ], shape=(100000,), dtype=float32), np.int64(7): array([-905.99304, -805.3275 , -782.7231 , ...,  797.17285,  814.96747,
        869.01685], shape=(100000,), dtype=float32), np.int64(8): array([-468.35513, -414.5815 , -400.18256, ...,  394.60294,  408.34427,
        465.80292], shape=(100000,), dtype=float32), np.int64(9): array([-251.13

100%|██████████| 1604/1604 [00:03<00:00, 496.93it/s]


printing V
{np.int64(2): array([-20034.992, -18218.387, -14097.58 , ...,  14483.667,  17931.166,
        20530.379], shape=(1604,), dtype=float32), np.int64(3): array([-10810.907, -10509.112, -10174.677, ...,   9099.767,   9140.594,
         9512.948], shape=(6416,), dtype=float32), np.int64(4): array([-5983.4688, -5945.6274, -5802.36  , ...,  5222.7627,  6135.2783,
        6482.0723], shape=(25664,), dtype=float32), np.int64(5): array([-3351.32  , -3325.228 , -3287.9463, ...,  2848.847 ,  2931.7986,
        2964.4297], shape=(100000,), dtype=float32), np.int64(6): array([-1739.9016, -1615.5277, -1607.1517, ...,  1571.7694,  1660.4429,
        1747.4661], shape=(100000,), dtype=float32), np.int64(7): array([-895.8082, -804.7751, -779.0027, ...,  771.4911,  800.6024,
        971.7964], shape=(100000,), dtype=float32), np.int64(8): array([-457.70154, -411.4987 , -395.43542, ...,  395.59308,  408.70312,
        480.3496 ], shape=(100000,), dtype=float32), np.int64(9): array([-251.77686, -